
# 配置文件介绍

ModelScope框架的核心思想之一是完全的配置化。在ModelHub的模型文件中，必然存在名为configuration.json或configuration.yaml的文件，这个文件
存储了模型拉起、推理、训练所需要的（几乎所有的）信息。因此，您可以直接将模型文件传入trainer中训练，或拷贝配置到新的模型中来复制一个SOTA的结果。

ModelScope中的配置文件存储了各式各样的配置信息，用以进行数据预处理、模型推理、训练和评估，确保模型推理、训练评估过程的可复现性。 

配置文件的格式支持`json`和`yaml`格式，模型仓库中默认的配置文件名称为`configuration.json`，ModelScope支持从模型仓库自动读取配置文件进行推理、训练，也支持使用本地配置文件方式进行相关操作。

# 读取和使用模型的配置文件

ModelScope有专门的API来读取配置文件。如果您需要读取模型的配置，可以按照如下代码进行：


In [1]:
from modelscope.utils.hub import read_config
cfg = read_config('damo/nlp_structbert_sentence-similarity_chinese-base')
print(cfg)



如果您想找到配置文件所在的文件目录，可以使用snapshot_download的能力：


In [1]:
from modelscope.utils.hub import snapshot_download
model_dir = snapshot_download('damo/nlp_structbert_sentence-similarity_chinese-base')
print(model_dir) # ~/.cache/modelscope/hub/*



一般来说，ModelScope在您使用训练和推理过程中会隐式地加载配置文件，您无需关心这一过程。
例如从远端模型仓库读取配置文件初始化模型预测：


In [1]:
from modelscope.pipelines import pipeline
word_segmentation = pipeline('word-segmentation')
input_str = '今天天气不错，适合出去游玩'
print(word_segmentation(input_str))



在训练和测试时，您可以从模型仓库下载配置文件到本地，也可以直接本地创建新的配置文件，指定本地配置路径进行使用：


In [1]:
from modelscope.trainers import build_trainer
from modelscope.msdatasets import MsDataset

model_id = 'damo/nlp_structbert_sentence-similarity_chinese-tiny'
# Ant Financial Question Matching Corpus (AFQMC) dataset
dataset_id = 'clue'

train_dataset = MsDataset.load('clue', subset_name='afqmc', split='train')
eval_dataset = MsDataset.load('clue', subset_name='afqmc', split='validation')

kwargs = dict(
    model=model_id,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    max_epochs=2,
    #cfg_name='your_configuration.json', #指向自定义的configuration文件
    work_dir='tmp')


trainer = build_trainer(default_args=kwargs)

trainer.train()



# 配置文件格式
配置文件主要包含如下一级字段:

- **framework**(必填)： 模型运行所需的框架， 例如pytorch，tensorflow，kaldi等。
- **task**(必填)： 模型所支持的任务类型，可以是str或者 str列表。
- pipeline(可选):  推理使用的pipeline类型。
- model (可选):  模型实例化相关参数配置，具体参数请直接参考对应模型库的configuration.json示例。
- dataset(可选):  训练评估过程中使用的数据集配置信息。
- preprocessor(可选): 训练评估过程中使用的预处理配置
- train(可选)：用以配置训练过程中的超参数，例如模型保存目录、训练轮数、优化器、学习率等参数。
- evaluation(可选): 用以配置评估过程中数据读取、评估指标等参数。


一个常见的配置文件示例如下
```json
{
    "framework": "pytorch",
    "task": "sentence-similarity",
    "pipeline": {
        "type": "sentence-similarity"
    },
    "preprocessor": {
        "type": "bert-seq-cls-tokenizer-finetune",
        "first_sequence": "sentence1",
        "second_sequence": "sentence2"
    },
    "model": {
        "type": "structbert",
        "attention_probs_dropout_prob": 0.1,
        "position_embedding_type": "absolute",
        "transformers_version": "4.6.0.dev0",
        "type_vocab_size": 2,
        "use_cache": true,
        "vocab_size": 21128
    },
    "dataset": {
        "train": {
            "name": "modelscope/afqmc_small",
            "split": "train"
        },
        "val": {
            "name": "modelscope/afqmc_small",
            "split": "val"
        }
    },
    "train": {
        "work_dir": "/tmp",
        "max_epochs": 10,
        "dataloader": {
            "batch_size_per_gpu": 2,
            "workers_per_gpu": 1
        },
        "optimizer": {
            "type": "SGD",
            "lr": 0.01
        },
        "lr_scheduler": {
            "type": "StepLR",
            "step_size": 2
        },
        "hooks": [{
            "type": "CheckpointHook",
            "interval": 1
        }]
    },
    "evaluation": {
        "dataloader": {
            "batch_size_per_gpu": 2,
            "workers_per_gpu": 1,
            "shuffle": false
        },
        "metrics": ["accuracy"]
    }
}
```

下面详细介绍model，dataset，preprocessor，train和evaluation等字段。
## model
model字段的二级字段**type**字段固定，表示模型注册名称， 其他参数一般根据算法提供者实现的模型初始化参数而定，例如类型为CustomModel的模型类的初始化函数定义如下：


In [1]:
@MODELS.register_module('sentence-similarity', 'custom-model')
class CustomModel(TorchModel):
    def __init__(self, 
                 attention_probs_dropout_prob=0.1,
                 gradient_checkpointing=False,
                 hidden_act='gelu',
                 hidden_dropout_prob=0.1,
                 hidden_size=768,
                 initializer_range=0.02,
                 intermediate_size=3072,
                 layer_norm_eps=1e-12,
                 max_position_embeddings=512,
                 num_attention_heads=12,
                 num_hidden_layers=12,
                 pad_token_id=0,
                 position_embedding_type='absolute',
                 type_vocab_size=2,
                 use_cache=True,
                 vocab_size=21128):
        pass


对应的configuration.json中model内容如下：
```json
{
  "model": {
    "type": "custom-model",
    "attention_probs_dropout_prob": 0.1,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_norm_eps": 1e-12,
    "max_position_embeddings": 512,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "pad_token_id": 0,
    "position_embedding_type": "absolute",
    "type_vocab_size": 2,
    "use_cache": true,
    "vocab_size": 21128
  }
}
```
可以看到，model中除type字段外，其他字段和__init__的入参是一一对应的。每种模型的model字段各不相同，具体请参考ModelScope的[模型文档]()。

但是，有几种情况不符合上面的信息：
- 部分模型是根据静态方法_instantiate()进行初始化的(当模型存在静态方法_instantiate的时候，否则会如同上例一样使用构造初始化)，在这种情况下，model中的字段对应的是_instantiate方法的入参
- 部分模型的codebase来自于transformers等其他框架，因此模型初始化参数在config.json中，这时候model的字段只有type是生效的，如果需要修改模型配置请修改config.json文件的内容，或在模型加载时动态传入

未来我们会移除config.json文件，使model配置完全融合到configuration.json中。

## dataset
dataset二级字段主要有train、val、test(可选)，用于指定训练集、验证集和测试集信息。
```json
{
  "dataset": {
    "train": {
      "name": "modelscope/afqmc_small",
      "split": "train"
    },
    "val": {
      "name": "modelscope/afqmc_small",
      "split": "val"
    }
  }
}
```
dataset各子key内部支持的信息如下：

| key值          |                                                   含义 |
|---------------|-----------------------------------------------------:|
| namespace     |     [对应MsDataset.load的namespace参数](../数据集/数据集的下载.ipynb) |
| target        |        [对应MsDataset.load的target参数](../数据集/数据集的下载.ipynb) |
| version       |       [对应MsDataset.load的version参数](../数据集/数据集的下载.ipynb) |
| hub           |           [对应MsDataset.load的hub参数](../数据集/数据集的下载.ipynb) |
| subset_name   |   [对应MsDataset.load的subset_name参数](../数据集/数据集的下载.ipynb) |
| split         |         [对应MsDataset.load的split参数](../数据集/数据集的下载.ipynb) |
| data_dir      |      [对应MsDataset.load的data_dir参数](../数据集/数据集的下载.ipynb) |
| data_files    |    [对应MsDataset.load的data_files参数](../数据集/数据集的下载.ipynb) |
| download_mode | [对应MsDataset.load的download_mode参数](../数据集/数据集的下载.ipynb) |
| data_files    |    [对应MsDataset.load的data_files参数](../数据集/数据集的下载.ipynb) |

一般来说，从ModelScope官网上下载的模型不带有dataset字段，这是因为训练的数据集多种多样，需要用户自行选择。
因此我们建议dataset字段在训练之前动态写入内存中的cfg中，或将原始configuration.json更新到其他目录再读取出来。

## preprocessor
预处理器的具体文档可以参考[这里](../ModelScope%20Library教程/数据的预处理.ipynb)
preprocessor的配置字段type以及参数和model类似， 根据对应preprocessor类的初始化函数参数决定。
```json
{
    "preprocessor": {
        "type": "sen-sim-tokenizer"
    }
}
```
上面的例子是NLP领域中，句子相似度的preprocessor。一般来说，每个模态的预处理器都有特定的参数，请查看对应模态的预处理器具体说明。

配置文件中的预处理器可以分为训练和测试两个，这时的配置文件类似这样：
```json
{
    "preprocessor": {
      "train": {
        "type": "sen-sim-tokenizer"
      },
      "val": {
        "type": "sen-sim-tokenizer"
      }
    }
}
```
在某些模态中训练和测试的预处理器是完全不同的。这样的设计有助于帮助不同的模型灵活预处理数据。对于代码读取部分，我们通过preprocessor_mode参数来控制读取的内容：


In [1]:
# train时读取train下面的配置，如果不存在train/val子key则读取preprocessor整体下面的配置，否则报错
Preprocessor.from_pretrained(model_name_or_path=model_dir, preprocessor_mode='train')
# eval/inference时读取val下面的配置，如果不存在train/val子key则读取preprocessor整体下面的配置，否则报错
Preprocessor.from_pretrained(model_name_or_path=model_dir, preprocessor_mode='eval')
Preprocessor.from_pretrained(model_name_or_path=model_dir, preprocessor_mode='inference')




#### train
训练参数一般由对应的trainer决定，官方默认提供的Trainer的配置一般有如下二级字段：
**注**： 当前提供的Trainer仅支持PyTorch框架训练，参数说明针对PyTorch框架

- work_dir：  训练模型保存目录
- max_epochs:  最大迭代轮次
- dataloader:   训练时Pytorch内部Dataloader的相关参数
- optimizer:  训练时所用的optimizer类型和参数，注意所有的参数和pytorch官方的torch.optim提供的[optimizer类](https://pytorch.org/docs/stable/optim.html)初始化参数一致
- lr_scheduler： 训练时所用的lr scheduler，参数与pytorch官方的[torch.optim.lr_scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)的接口一致
- hooks:  所有回调函数的配置， 支持配置不同回调函数，详细配置可以参考[回调函数机制介绍](./回调函数机制详解.ipynb)
```json
{
  "train": {
    "work_dir": "/tmp",
    "max_epochs": 10,
    "dataloader": {
      "batch_size_per_gpu": 2,
      "workers_per_gpu": 1
    },
    "optimizer": {
      "type": "SGD",
      "lr": 0.01,
      "options": {
        "grad_clip": {
          "max_norm": 2.0
        }
      }
    },
    "lr_scheduler": {
      "type": "StepLR",
      "step_size": 2,
      "options": {
        "warmup": {
          "type": "LinearWarmup",
          "warmup_iters": 2
        }
      }
    },
    "hooks": [
      {
        "type": "CheckpointHook",
        "interval": 1
      },
      {
        "type": "TextLoggerHook",
        "interval": 1
      },
      {
        "type": "IterTimerHook"
      },
      {
        "type": "EvaluationHook",
        "interval": 1
      }
    ]
  }
}
```

在训练时，optimizer和lr_scheduler的一些参数需要运行时写入。比如需要lambda表达式作为入参，或者需要填入训练total_iters等情况。这种情况请具体参考[训练文档](../ModelScope%20Library教程/模型的训练Train.ipynb)。


#### evaluation
评估部分常用的字段如下

- dataloader:  评估时Pytorch内部Dataloader的相关参数
- metrics： 评估所用的metric名称，  当前支持的有
   - accuracy
   - image-denoise-metric
   - image-ins-seg-coco-metric
   - seq-cls-metric
   - token-cls-metric
   - text-gen-metric
   - image-color-enhance-metric
   - image-portrait-enhancement-metric
 
Metric的具体介绍请参考[这里](../ModelScope%20Library教程/模型的评估.ipynb)。
```json
{
  "evaluation": {
    "dataloader": {
      "batch_size_per_gpu": 2,
      "workers_per_gpu": 1,
      "shuffle": false
    },
    "metrics": [
      "accuracy"
    ]
  }
}
```
